# Recurrent Residual Convolutional Neural Network

### Simon Laurent Lebailly, 2549365, s9sileba@teams.uni-saarland.de
### Christian Mathieu Schmidt, 2537621, s9cmscmi@teams.uni-saarland.de

## Import libaries

In [1]:
import os
from os.path import join as pjoin
import collections
import json
import torch
import imageio
import numpy as np
import scipy.misc as m
import scipy.io as io
import matplotlib.pyplot as plt
import glob

from PIL import Image
from tqdm import tqdm

## Use CUDA if possible

In [2]:
#Check if CUDA is available, if not use the CPU.
train_on_GPU = torch.cuda.is_available()
device = torch.device('cuda' if train_on_GPU else 'cpu')

if train_on_GPU:
    print('CUDA available!')
else:
    print('CUDA not available!')

CUDA not available!


## Define Hyperparameters

In [3]:
batch_size = 12
epochs = 10

learning_rate = 0.0001

## Import dataset Cityscapes

In [4]:
import torchvision
from torchvision import transforms
from torchvision import datasets
from torch.utils import data


#Define normalization for dataset
normalize = transforms.Normalize(
    mean = [0.485, 0.456, 0.406],
    std = [0.229, 0.224, 0.225]
)

#Define transformation for train, validation and test dataset
transform = transforms.Compose([
    transforms.Resize((256,512)),
    transforms.ToTensor(),
    normalize
])

target_transform = transforms.Compose([
    transforms.Resize((256,512))
])


#Tensor to image
def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


#Path of dataset
root_path = 'C:/Users/chris/Documents/NN-Projekt/Cityscapes_dataset/Cityscapes'


#Import dataset for training
train_set = datasets.Cityscapes(root=root_path, split='train', mode='fine', target_type='instance', transform=transform, target_transform=target_transform)
quantity_train = len(train_set)
print('Quantity training data: '+ str(quantity_train))

#Prepare training dataset for NN
train_loader = data.DataLoader(train_set, batch_size=batch_size, shuffle=True)


#Import dataset for validation
validation_set = datasets.Cityscapes(root=root_path, split='val', mode='fine', target_type='instance', transform=transform, target_transform=target_transform)
quantity_validation = len(validation_set)
print('Quantity validation data: '+ str(quantity_validation))

#Prepare validation dataset for NN
validation_loader = data.DataLoader(validation_set, batch_size=batch_size, shuffle=True)


#Import dataset for testing
test_set = datasets.Cityscapes(root=root_path, split='test', mode='fine', target_type='instance', transform=transform, target_transform=target_transform)
quantity_test = len(test_set)
print('Quantity testing data: '+ str(quantity_test))

#Prepare test dataset for NN
test_loader = data.DataLoader(test_set, batch_size=batch_size, shuffle=False)

Quantity training data: 2975
Quantity validation data: 500
Quantity testing data: 1525


In [5]:
#Printouts for testing train set

#print(train_loader)
#print(validation_loader)
#print(test_loader)

#img, segm = train_set[1]
#print(img)
#print(img.size())
#imshow(torchvision.utils.make_grid(img))
#print(segm)
#segm.show()
#print(type(train_set))
#print(train_set)

## Define R2U-Net model

In [6]:
import torch.nn as nn

class R2U_Net(nn.Module):
    def __init__(self):
        super(R2U_Net, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 30, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
            nn.Softmax()
        )


    def forward(self, x):
        out = self.conv(x)
        return out

In [7]:
#Load model to device
r2u_net = R2U_Net().to(device)

## Define Criterion and Optimizer

In [8]:
# loss function
criterion = nn.CrossEntropyLoss().to(device)

# optimizer variable
opt = torch.optim.Adam(r2u_net.parameters(), lr=learning_rate)

## Load model

In [9]:
if os.path.isfile('task_2_model_parameters.pt'):
    r2u_net = torch.load('task_2_model_parameters.pt')

## Train loop

In [10]:
for epo in range(epochs):
    trainloader_loop = tqdm(train_loader)

    for i, data in enumerate(trainloader_loop):
        img, smg = data[0]
        break

    torch.save(r2u_net, 'task_2_model_parameters.pt')
    break

print("Finished training!")

  0%|          | 0/248 [00:01<?, ?it/s]


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'PIL.Image.Image'>

## Evaluation

In [ ]:
from scipy import spatial
import sklearn.metrics as metrics


def one_row(ground_truth, prediction):
    truth = prediction.argmax(dim=1).view(1,-1)
    truth = np.squeeze(truth.numpy())

    pred = ground_truth.view(1,-1)
    pred = np.squeeze(pred.numpy())

    return truth.astype(float), pred.astype(float)


def evaluate(ground_truth, predictions):
    ground_truth2 = ground_truth.copy()
    predictions2 = predictions.copy()

    f1_score = 0
    auc_score = 0
    dice_coefficient = 0

    listlen = min(len(ground_truth2),len(predictions2))

    for l in range(0, listlen):
        x = ground_truth2[l].cpu()
        y = predictions2[l].cpu()

        transform_tensors = one_row(x, y)

        f1_score += metrics.f1_score(transform_tensors[0], transform_tensors[1],average=None)
        auc_score += metrics.roc_auc_score(transform_tensors[0], transform_tensors[1], average=None, multi_class='ovo')
        dice_coefficient += abs(spatial.distance.dice(transform_tensors[0], transform_tensors[1], w=None))

    f1_score = f1_score/listlen
    auc_score = auc_score/listlen
    dice_coefficient = dice_coefficient/listlen

    return f1_score, auc_score, dice_coefficient

## Evaluation plots